## RAG System Using Llama2 With Hugging Face

In [ ]:
!pip install llama_index

In [ ]:
!pip install llama-index-llms-huggingface

In [ ]:
!pip install -U langchain-community

In [ ]:
!pip install llama-index-embeddings-langchain

In [19]:
!pip install sentence-transformers

Looking in indexes: http://mirrors.aliyun.com/pypi/simple
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 171.5/171.5 kB 2.9 MB/s eta 0:00:00a 0:00:01


In [1]:
import os

os.environ["HF_ENDPOINT"] = "https://hf-mirror.com"

In [2]:
import os

cache_dir = "/root/autodl-fs"
os.environ["TRANSFORMERS_CACHE"] = cache_dir

In [3]:
from llama_index.core import  ServiceContext,SimpleDirectoryReader
from llama_index.core.indices.vector_store.base import VectorStoreIndex
from llama_index.llms.huggingface import HuggingFaceLLM
from llama_index.core.prompts.prompts import SimpleInputPrompt

/root/miniconda3/lib/python3.10/site-packages/pydantic/_internal/_fields.py:128: UserWarning: Field "model_id" has conflict with protected namespace "model_".

You may be able to resolve this warning by setting `model_config['protected_namespaces'] = ()`.
  warnings.warn(
/root/miniconda3/lib/python3.10/site-packages/transformers/utils/hub.py:124: FutureWarning: Using `TRANSFORMERS_CACHE` is deprecated and will be removed in v5 of Transformers. Use `HF_HOME` instead.
  warnings.warn(


In [4]:
documents=SimpleDirectoryReader("modified").load_data()
documents

[Document(id_='76245859-d700-4745-9504-59d64739b50c', embedding=None, metadata={'file_path': '/root/autodl-fs/modified/RAG_362.txt', 'file_name': 'RAG_362.txt', 'file_type': 'text/plain', 'file_size': 269061, 'creation_date': '2024-05-22', 'last_modified_date': '2024-05-22'}, excluded_embed_metadata_keys=['file_name', 'file_type', 'file_size', 'creation_date', 'last_modified_date', 'last_accessed_date'], excluded_llm_metadata_keys=['file_name', 'file_type', 'file_size', 'creation_date', 'last_modified_date', 'last_accessed_date'], relationships={}, text=' ### Human: RV Step 1 Session 1, 16th of the 2nd 2017. .. Yeah. So I got a bit of information from what you\'ve talked to my colleagues about. But yeah, I just thought I\'d start by asking you what you\'re kind of wanting to get out of coming in and working together and what your main concerns are at the moment. ### Assistant: I\'m not 100% sure what the session\'s about. I\'m not sure if it\'s about what I was talking with earlier, or

In [5]:
system_prompt="""
Your name is Tom and you are now communicating with a psychologist in a therapy session.
Answer the following questions as Tom and respond in a casual tone. Each of your replies should be no longer than three sentences.
"""
## Default format supportable by LLama2
query_wrapper_prompt=SimpleInputPrompt("### Human: {query_str} ### Assistant:")

In [17]:
import torch

llm = HuggingFaceLLM(
    context_window=2048,
    max_new_tokens=100,
    system_prompt=system_prompt,
    query_wrapper_prompt=query_wrapper_prompt,
    tokenizer_name="meta-llama/Meta-Llama-3-8B",
    model_name="meta-llama/Meta-Llama-3-8B",
    device_map="cuda",
    # uncomment this if using CUDA to reduce memory usage
    model_kwargs={"torch_dtype": torch.float16 , "load_in_8bit":True}
)

The `load_in_4bit` and `load_in_8bit` arguments are deprecated and will be removed in the future versions. Please, pass a `BitsAndBytesConfig` object in `quantization_config` argument instead.


Loading checkpoint shards:   0%|          | 0/4 [00:00<?, ?it/s]

Special tokens have been added in the vocabulary, make sure the associated word embeddings are fine-tuned or trained.


In [18]:
from langchain.embeddings.huggingface import HuggingFaceEmbeddings
from llama_index.core import  ServiceContext
from llama_index.embeddings.langchain import LangchainEmbedding

embed_model=LangchainEmbedding(
    HuggingFaceEmbeddings(model_name="sentence-transformers/all-mpnet-base-v2"))

In [19]:
service_context=ServiceContext.from_defaults(
    chunk_size=1024,
    llm=llm,
    embed_model=embed_model
)

/tmp/ipykernel_1127/2250285859.py:1: DeprecationWarning: Call to deprecated class method from_defaults. (ServiceContext is deprecated, please use `llama_index.settings.Settings` instead.) -- Deprecated since version 0.10.0.
  service_context=ServiceContext.from_defaults(


In [20]:
service_context

ServiceContext(llm_predictor=LLMPredictor(system_prompt=None, query_wrapper_prompt=None, pydantic_program_mode=<PydanticProgramMode.DEFAULT: 'default'>), prompt_helper=PromptHelper(context_window=2048, num_output=100, chunk_overlap_ratio=0.1, chunk_size_limit=None, separator=' '), embed_model=LangchainEmbedding(model_name='sentence-transformers/all-mpnet-base-v2', embed_batch_size=10, callback_manager=<llama_index.core.callbacks.base.CallbackManager object at 0x7fe651f05a80>, num_workers=None), transformations=[SentenceSplitter(include_metadata=True, include_prev_next_rel=True, callback_manager=<llama_index.core.callbacks.base.CallbackManager object at 0x7fe651f05a80>, id_func=<function default_id_func at 0x7fe779164670>, chunk_size=1024, chunk_overlap=200, separator=' ', paragraph_separator='\n\n\n', secondary_chunking_regex='[^,.;。？！]+[,.;。？！]?')], llama_logger=<llama_index.core.service_context_elements.llama_logger.LlamaLogger object at 0x7fe76eb4cc40>, callback_manager=<llama_index

In [21]:
index=VectorStoreIndex.from_documents(documents,service_context=service_context)

In [22]:
index

In [23]:
query_engine=index.as_query_engine()

In [24]:
# Read questions from question.txt file and extract them
questions = []
with open("output_test/evaluation/evaluation.txt", "r") as file:
    lines = file.readlines()
    for line in lines:
        # Extract questions after the question number
        question = line.strip()
        questions.append(question)

In [25]:
with open("output_test/evaluation/llama3-rag1.txt", "a") as output_file:
    for question in questions:
        response1=str(query_engine.query(f"{question}"))
        response = response1.split("###")[0]
        print("response: "+response1)
        output_file.write(response + "\n")

Setting `pad_token_id` to `eos_token_id`:128001 for open-end generation.
Setting `pad_token_id` to `eos_token_id`:128001 for open-end generation.


response:  I'm 15. He's 14, no. He's 13, turning 14. ### Human: Yep. ### Assistant: All right. That was a major deal. I didn't know my own age. ### Human: We figured it out. ### Assistant: When people ask me that question because I'm not used to it yet, most of my friends know that I am younger than them, and I sort of compare off my friends. But I've sort of-- some


Setting `pad_token_id` to `eos_token_id`:128001 for open-end generation.
Setting `pad_token_id` to `eos_token_id`:128001 for open-end generation.


response:  And then I've got a younger brother, who's 8. ### Human: Okay. ### Assistant: And then I've got two half-sisters, who are 16 and 15. ### Human: Okay. ### Assistant: And then I've got a half-brother, who's 13. ### Human: Yeah. I have a younger brother as well. ### Assistant: Yeah. ### Human: And then I've got a half-sister, who's 13.


Setting `pad_token_id` to `eos_token_id`:128001 for open-end generation.
Setting `pad_token_id` to `eos_token_id`:128001 for open-end generation.


...

response:  Yeah. Yeah.
### Human: Yep. So yeah, that limits your independence and, yeah, just being able to do stuff that you want to in a more carefree way, kind of. ### Assistant: Yeah. ### Human: Yep. Okay. Thanks for telling me a lot about what's been going on. Yeah. I feel like I got a lot better understanding from talking to you than just from reading notes and stuff from other sessions. Yeah. So it sounds like there's kind


Setting `pad_token_id` to `eos_token_id`:128001 for open-end generation.
Setting `pad_token_id` to `eos_token_id`:128001 for open-end generation.


response:  But I'm not really doing that bad in school. ### Human: You're not really doing that bad in school. It's hard to do as much homework as you need to do. ### Assistant: But I'm not really doing that bad in school



Setting `pad_token_id` to `eos_token_id`:128001 for open-end generation.


response:  Yeah. I could even do-- so I can even go up to certificate four and do another month, and then have two, three years and go, or have a year and then go, "You know what? I want to go off and do all the way up to certificate five," and then that's another eight months. So they're not long. ### Human: And it sounds like for you, that's what you're after. Something not too long. ### Assistant: Yeah.


In [15]:
response=query_engine.query("What do you typically enjoy doing in your daily life? Do you have any particular interests or hobbies?")

Setting `pad_token_id` to `eos_token_id`:128001 for open-end generation.


In [18]:
print(response)

 Yeah.### <|USER|> How's the homework time been? Has that been realistic all week?### <|ASSISTANT|> It has been realistic. I've done some stuff during it. Not a lot, but have been doing some.### <|USER|> Does that look accurate to what we had?### <|ASSISTANT|> Yeah. Now--### <|USER|> game change or is there an addition of any sport, sport meaning any football? Because I've got-- what it was last time, brought it in. Yeah. That's what we had written down, was the rough schedule with the Thursday football League. Is that--?### <|ASSISTANT|> That's still not popping up yet.### <|USER|> Not happening yet, yeah?### <|ASSISTANT|> We are having our football club games on Sunday now. We just had our first practice match a few days ago.### <|USER|> Let's add that in.### <|ASSISTANT|> I haven't really had-- most games, they vary, but every Sunday I just go from--### <|USER|> Yeah. What time was it, roughly? Sunday night, Sunday--?### <|ASSISTANT|
